In [19]:
import numpy as np

class Queen:
    def __init__(self, N):
        self.N = N
        self.queen_loc = dict()
        self.initialize = False
        self.chess_board = [[0]*self.N for _ in range(self.N)]

    def add_queen(self):
        if self.initialize == False:
            number_Q = 0
            while True:
                flag = 0
                r = np.random.randint(self.N)
                c = np.random.randint(self.N)
                for q in self.queen_loc:
                    row, col = self.queen_loc[q]
                    if (r==row and c==col) or (c==col):
                        flag = 1
                if flag == 0:
                    Q = f"Q{number_Q}"
                    if Q not in self.queen_loc:
                        self.queen_loc[Q] = []
                    self.queen_loc[Q].append(r)
                    self.queen_loc[Q].append(c)
                    self.chess_board[r][c] = Q
                    number_Q += 1
                if number_Q == self.N:
                    break
            self.initialize = True

    def get_neighbor(self, row, col):
        neighbor = []
        if 0<=row-1<self.N and self.chess_board[row-1][col]==0:
            neighbor.append([row-1,col])
        if 0<=row+1<self.N and self.chess_board[row+1][col]==0:
            neighbor.append([row+1,col])
        return neighbor

    def print_Queen(self):
        print(self.chess_board)
        for Q in self.queen_loc:
            print(f'{Q}->{self.queen_loc[Q]}')

def conflict(r1,c1,r2,c2):
    if r1 == r2:
        return True
    if c1 == c2:
        return True
    if r1+c1 == r2+c2:
        return True
    if r1-c1 == r2-c2:
        return True
    return False

def get_conflict(Q,state):
    count = 0
    for q in state:
        if q is not  Q:
            r1, c1 = state[Q]
            r2, c2 = state[q]
            if conflict(r1,c1,r2,c2):
                count += 1
    return count

def calc_cost(state):
    cost = 0
    max = -999
    maxQ = None
    for Q in state:
        q_cost = get_conflict(Q,state)
        cost += q_cost
        if q_cost>max:
            max = q_cost
            maxQ = Q
    return cost//2,max,maxQ

def goal_test(state):
    if calc_cost(state)[0] == 0:
        return True
    else:
        return False

In [20]:
def simulated_annealing(queen, temp, alpha):
    current_state = queen.queen_loc.copy()
    while temp > 0.001:  # stopping criteria
        neighbors = []
        for Q in current_state:
            r, c = current_state[Q]
            for new_pos in queen.get_neighbor(r, c):
                new_state = current_state.copy()
                new_state[Q] = new_pos
                neighbors.append((new_state, calc_cost(new_state)[0]))
        if not neighbors:  # reached a local maximum - the get_neighbor function returned an empty list
            break
        neighbors.sort(key=lambda x: x[1])  # sorts the list of neighbors
        next_state = neighbors[0][0]
        delta_cost = calc_cost(next_state)[0] - calc_cost(current_state)[0]
        if delta_cost < 0 or np.random.uniform(0, 1) < np.exp(-delta_cost / temp):
            current_state = next_state
        temp *= alpha  # decrease the temperature
    return current_state

# Main Code
queen = Queen(4)
queen.add_queen()
queen.print_Queen()

solution = simulated_annealing(queen, temp=1000, alpha=0.95)
print("Solution:")
for Q in solution:
    print(f'{Q}->{solution[Q]}')

[['Q3', 0, 'Q2', 'Q0'], [0, 0, 0, 0], [0, 0, 0, 0], [0, 'Q1', 0, 0]]
Q0->[0, 3]
Q1->[3, 1]
Q2->[0, 2]
Q3->[0, 0]
Solution:
Q0->[0, 3]
Q1->[3, 1]
Q2->[2, 2]
Q3->[1, 0]
